# Visualizing Flight Test Data Interactively With Open Source Tools
---
## Society of Flight Test Engineers 49th Annual International Symposium
### 9 October 2018, Savannah GA
### Luke Starnes (GTRI)

# Agenda
* OSS Value Proposition
* ADS-B Background
* Tooling Overview
* Examples

# OSS Value Proposition


* Proprietary data analysis tools are expensive and create “vendor lock”
* Walled Garden

 ![](images/walled_garden.jpg)

# OSS Value Proposition

* open source tools are a superior choice for today’s flight test analysis problems
* open interfaces
* widespread compatibility (community of interoperable tools)
* seamless migration between tools (no “vendor lock”)
* flexibility and agility

# Open Flight Data as Lens for Talking OSS Tooling


<div align="center"><table><tr><td><img src='images/adsb.png'></td><td><img src='images/lens.png'></td><td><img src='images/osi_logo.png'></td></tr></table></div>


# ADS-B Background
* Automatic Dependent Surveillance-Broadcast
* Airfract system for broadcasting identification and position data
* Facilitated by uibiquity of GPS
* Driven by cost of maintaining ATC radars
* ADSB mandated in US starting Jan 1, 2020
 * required for aircraft operating about 10k', around airports, or off Gulf of Mexico
* European mandate starts Jan 1, 2019

* ADSB is Line of Sight - requires network of ground stations to recieve reports (min ~100NM)

<div align="center"><img src="images/adsb_ground_stations.png"></div>

* Transmissions are unencrypted
Thus a preponderance of...

<div align="center"><img src="images/prostick.jpg" height=30% width=30%></div>

* ADSB is Line of Sight - requires network of ground stations to recieve reports (min ~100NM)
* Transmissions are unencrypted
* Thus a preponderance of... 

<div align="center"><img src="images/planefinder.png"></div>
<div align="center"><sup>Source: [planefinder.net](https://planefinder.net/)</sup></div>

Other simlar sites: include [flightradar24.com](https://www.flightradar24.com/), [flightaware.com](https://flightaware.com/), and [adsbexchange.com](https://www.adsbexchange.com/).

* ADSB-B Exchange ([adsbexchange.com](https://www.adsbexchange.com/)) provides public access to their worldwide dataset (begins June 9, 2016)
<div align="center"><img src="images/adsbexchange_logo_full.png"></div>
* Data made available as JSON
* Each day is a single zip file with 1,440 JSON files (1 file per minute)

# OSS Tool Stack
* __Hierarchical Data Format 5 (HDF5)__ - multiplatform, effient, fast data storage format, metadata support
* __Pandas__ - robust tool for accessing, transforming, and analyzing tabular data
* __Luigi__ - pipelining tool for managing complex pipelines with inter-dependent steps
* __Jupyter__ - (*this*) web tool for integrating code, documentation, and visualization into narrative notebook
* __Bokeh__ - browser-based interactive visualization tool
* __Datashader__ - plotting tool for visualizing large datasets (points >> pixels)

# Pandas

In [2]:
import pandas as pd
import os

In [3]:
%%time
row_len = 500_000
h5_dir = r'c:\adsb'
h5_file = os.path.join(h5_dir, '2018-06-16.h5')
pickle_name = f'{os.path.basename(h5_file)}-{row_len}.p'
pickle_path = os.path.join(os.getcwd(), 'data', pickle_name)
if os.path.exists(pickle_path):
    df = pd.read_pickle(pickle_path)
else:
    with pd.HDFStore(h5_file, mode='r') as store:
        df = store.select('data', stop = row_len, columns=['Man', 'Icao', 'Type', 'Op'])
    main_ops = ['Southwest', 'American', 'Delta', 'SkyWest', 'Air Canada', 'Alaska', 
                'Virgin', 'United','JetBlue', 'Spirit', 'Frontier', 'Wells Fargo']
    for o in main_ops:
        df.loc[df.Op.fillna('-').str.lower().str.contains(o.lower()), 'Op'] = o
    df.to_pickle(pickle_path)

Wall time: 644 ms


In [4]:
print(df.shape)
df.dropna(how='any').head()

(500000, 4)


,Man,Icao,Type,Op
1,Raytheon Aircraft Company,A3286B,BE40,"MOSER AVIATION LLC - ENGLEWOOD, CO"
3,Boeing,AB1FFE,B739,Delta
6,Robinson,A56D30,R44,Robinson Helicopter Company
10,Airbus,424356,A320,Aeroflot Russian Airlines
13,McDonnell Douglas,AD8563,MD83,Wells Fargo


In [51]:
df['Man'].value_counts()[:10]

Boeing                          120932
Airbus                           92420
Embraer                          27713
Bombardier                       26085
Cessna                            9963
McDonnell Douglas                 5777
Gulfstream Aerospace              2578
Beech                             2491
Piper                             2365
Avions de Transport Regional      2350
Name: Man, dtype: int64

In [53]:
df.groupby('Op').agg({'Icao': pd.Series.nunique}).sort_values('Icao', ascending=False)[:10]

,Icao
Op,
American,649
Delta,573
United,537
Southwest,483
Wells Fargo,327
Private,234
Air Canada,226
JetBlue,150
Virgin,132


In [47]:
airlines_filter = df['Op'].isin(df.Op.value_counts().index[:10])
table = df[airlines_filter].groupby(['Op','Type']).agg({'Icao': pd.Series.nunique}).unstack().T
table['Total'] = table.sum(skipna=True, axis=1).map(int)
table.sort_values('Total', ascending=False).fillna('')[:10]

Op        Air Canada American Delta JetBlue Private SkyWest Southwest United  \
     Type                                                                      
Icao B738                 172    52                               142     48   
     B737                         4                               317     17   
     A320         36       24    37      83                               57   
     A321         12      141    40      37                                    
     A319         22       55    33                                       45   
     B739                        64                                       80   
     CRJ9         11        7    60                      10                    
     E145                  50                                              7   
     B752                   9    50                                       29   
     CRJ7                  13    10                      38                9   

Op        Virgin Wells Fargo  Total  
     Type                            
Icao B738     53          21    488  
     B737      2          18    358  
     A320     35          10    282  
     A321                  4    234  
     A319      7          13    175  
     B739                  1    145  
     CRJ9                 25    113  
     E145                 36     93  
     B752                  4     92  
     CRJ7                 21     91

# Bokeh

# Why Visualization is important
<div align="center"><img src="images/anscombe's_quartet.png"></div>
<div align="center">Anscombe's quartet - dataset consisting of four sets of points which are all statistically similar, but visually varied.</div>


# Bokeh Examples
* [Heat Maps](/notebooks/GitHub/sfte2018-adsb/Bokeh - Heat Map.ipynb)
* [Flight Data with Map Tiles](/notebooks/GitHub/sfte2018-adsb/Flight Data with Map Tiles.ipynb)
* [Flight Data with Google Maps](/notebooks/GitHub/sfte2018-adsb/Flight Data with Google Maps.ipynb)

# Datashader - The Why

<div align="center"><img src="images/datashader-plotting-pitfalls.png"></div>


* overplotting - occlusion
* oversatuation - after employing transparency, you still have cases where details are not fully discernable. oversaturation obscures spatial differences in density
* undersampling - make dots really small. But this can hide important detail
* undersaturation - by trying to fix above problems, its easy to end up with case of 
* Underutinzied range - the need to appropriatly apply color ad a function of count in a way to relays most info
* Nonuniform colormapping - the improtance of using color palett that shows detail

# Datashader Examples
* [Worldwide Viz with Datashades](/notebooks/GitHub/sfte2018-adsb/Worldwide Viz with Datashader.ipynb)
* [Interactive Datashader](/notebooks/GitHub/sfte2018-adsb/Interactive Datashader.ipynb)

# Conclusion

* open source tools are a superior choice for today’s flight test analysis problems
* open interfaces
* widespread compatibility (community of interoperable tools)
* seamless migration between tools (no “vendor lock”)
* flexibility and agility

### Slides / Notebooks available here:

* https://github.com/slstarnes/sfte2018-adsb